### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__New York City Airbnb Open Data__  
  URL:  
  ・https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/  
  DATA:  
  ・AB_NYC_2019.csv

In [ ]:
%matplotlib inline

import branca.colormap as cm
import folium
from folium.plugins import HeatMap
import pandas as pd
import seaborn as sns

### 1. 行、列の指定による抽出

Airbnbの掲載データをロード。

In [ ]:
ab_listings_df = pd.read_csv('data/AB_NYC_2019.csv')

len(ab_listings_df)

データの中身を確認。

In [ ]:
ab_listings_df.head()

データの型と欠損を確認。

In [ ]:
ab_listings_df.info()

#### 1-1. 行番号、列番号の指定
行番号、列番号の指定には`pandas.DataFrame.iloc`属性を利用。  
`pandas.DataFrame.iloc`はメソッドではないので、角括弧に番号を指定します。

３行目を抽出。

In [ ]:
ab_listings_df.iloc[2] # 番号は0から始まる

2〜4行目を抽出。  
「（始まりの番号）:（終わりの番号+1）」と指定。

In [ ]:
ab_listings_df.iloc[1:4] # 番号は0から始まる

6列目を抽出。  
行を指定しない場合は「:」（コロン）とします。

In [ ]:
ab_listings_df.iloc[:,5] # 番号は0から始まる

#### 1-2. 行ラベル、列ラベルの指定
行番号、列番号の指定には`pandas.DataFrame.loc`属性を利用。  
`pandas.DataFrame.loc`もメソッドではないので、角括弧に番号を指定します。

行ラベルを作るために`neighbourhood`でグループ分けします。

In [ ]:
ab_listings_group_by_neighbourhood_df = ab_listings_df.groupby('neighbourhood').mean().round(2) # とりあえず平均。

ab_listings_group_by_neighbourhood_df.head()

任意の行ラベルで抽出。

In [ ]:
ab_listings_group_by_neighbourhood_df.loc['Arden Heights']

任意の列ラベルで抽出。  
行を指定しない場合は「:」を置きます。

In [ ]:
ab_listings_df.loc[:,'neighbourhood']

列ラベルで抽出する場合は、`loc`を使わずにそのまま角括弧で指定できます。

In [ ]:
ab_listings_df['neighbourhood']

### 2. 条件指定による抽出

たとえば、`price`列に対して「100未満」という比較をすると、データの行数分のブール値が返ってきます。

In [ ]:
price_under_100_bools = ab_listings_df['price'] < 100

price_under_100_bools

これを元のデータフレームの角括弧に指定すると、Trueに該当する行のみが抽出されます。

In [ ]:
price_under_100_df = ab_listings_df[price_under_100_bools]

price_under_100_df

価格の最大値が100ドル以下であることを確認。

In [ ]:
price_under_100_df['price'].max()

`pandas.DataFrame.loc`でも同じ結果が得られます。

In [ ]:
price_under_100_loc_df = ab_listings_df.loc[price_under_100_bools]

price_under_100_loc_df

`pandas.DataFrame.query()`を利用することで、SQLのように書くこともできます。

In [ ]:
ab_listings_df.query('host_name == "LisaRoxanne"')

範囲の指定もできます。

In [ ]:
ab_listings_df.query('365 < minimum_nights < 500')

これを`query()`を使わないでやると、なかなか面倒です。

In [ ]:
### チャレンジしてみよう！！ ###

### 3. ランダム抽出

ランダム抽出は`pandas.DataFrame.sample()`を使えば簡単です。

ここでは説明しやすいよう、毎回同じ結果となるように`random_state`を指定してあります。

参考）  
生命、宇宙、そして万物についての究極の疑問の答え  
https://ja.wikipedia.org/wiki/生命、宇宙、そして万物についての究極の疑問の答え

In [ ]:
ab_listings_sample_df = ab_listings_df.sample(frac=0.2, random_state=42) # 全体の20%を抽出

len(ab_listings_sample_df) / len(ab_listings_df)

ただし、単純に`pandas.DataFrame.sample()`で抽出してしまうと問題がある場合もあります。

たとえばあるホスト（`host_id`=2787）は複数の物件を掲載していますが、その中の一部しか抽出されません。  
この場合、ホストあたりの物件数に焦点を当てた分析が正しくおこなえません。

In [ ]:
print("ホスト（host_id=2787）の物件数")

print("・抽出前: ", len(ab_listings_df.query('host_id == 2787')), "件")

print("・抽出後: ", len(ab_listings_sample_df.query('host_id == 2787')), "件")

この場合、ホスト単位でランダム抽出をおこないます。
 
ただし、最終的に抽出される件数が若干ずれる可能性があります。

In [ ]:
host_id_sample = pd.Series(ab_listings_df['host_id'].unique()).sample(frac=0.2, random_state=42)

ab_listings_sample_revised_df = ab_listings_df.query('host_id in @host_id_sample') # @マークで変数にアクセス可

ab_listings_sample_revised_df.head()

データ数がおおよそ20%になっていることを確認。

In [ ]:
len(ab_listings_sample_revised_df) / len(ab_listings_df)

先ほどのホスト（`host_id`=2787）の物件数を確認します。  
全件か、0件になっていれば成功です。

`sample()`の引数`random_state`を数字を変えてもどちらかになるはずです。

In [ ]:
print("ホスト（host_id=2787）の物件数")

print("・抽出前: ", len(ab_listings_df.query('host_id == 2787')), "件")

print("・抽出後: ", len(ab_listings_sample_revised_df.query('host_id == 2787')), "件")

### 4. 地図に表示
せっかく緯度経度があるので地図上にデータを表示してみます。

まず、件数が多いので、`folium.plugins.HeatMap`を使って分布を確認します。   
どんなことが読み取れますか？

In [ ]:
# ニューヨークの座標
new_york_city_coordinates = [40.7128, -74.0060]

# 全データの座標を抽出し、リストに変換
ab_listings_coords = ab_listings_df[['latitude', 'longitude']].values.tolist()

# 地図を描画
m = folium.Map(location=new_york_city_coordinates, zoom_start=9.5)

HeatMap(ab_listings_coords, radius=5, blur=5).add_to(m)

m

次は任意の500件を抽出し、それぞれの座標上に価格で色分けした円を描きます。

まずヒストグラムで価格の分布を確認。

In [ ]:
sns.histplot(ab_listings_df['price'], kde=False)

極端な外れ値があるようなので、1,500ドル未満のみを抽出してもう一度確認。

In [ ]:
ab_listings_no_too_expensive_df = ab_listings_df[ab_listings_df['price'] < 1500]

sns.histplot(ab_listings_no_too_expensive_df['price'], kde=False)

今度は500件をランダム抽出して分布を確認。  
`sample()`に引数`n=500`を指定します。

分布が大きく変わっていなければOKです。

In [ ]:
ab_listings_no_too_expensive_sample_df = ab_listings_no_too_expensive_df.sample(n=500)

sns.histplot(ab_listings_no_too_expensive_sample_df['price'], kde=False)

地図上にデータを表示。  
どんなことが読み取れますか？

In [ ]:
# ヒストグラムの最大・最小を見ながらカラーマップを設定
colormap = cm.LinearColormap(colors=['blue','red'], vmin=0,vmax=1000)

m = folium.Map(location=new_york_city_coordinates, zoom_start=9.5)

for index, row in ab_listings_no_too_expensive_sample_df.iterrows():
    location = (row['latitude'], row['longitude'])
    color = colormap(row['price'])
    popup_message_html = f"<p>\"{row['name']}\"</p><p>host: {row['host_name']}<p>price: ${row['price']:,}</p></p>"
    popup = folium.Popup(folium.IFrame(popup_message_html), min_width=400, max_width=400)

    folium.Circle(location=location,
                  radius=10,
                  color=color,
                  fill=True,
                  fill_opacity=1,
                  popup=popup).add_to(m)

m